In [7]:
%pip install -q transformers datasets nltk spacy seaborn matplotlib Wikipedia-API

Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers import AutoModel, AutoTokenizer
import torch

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [6]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large').to(device)

In [8]:
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia('MyProjectName (merlin@example.com)', 'ru')
page = wiki_wiki.page('Россия')

text = page.text

In [10]:
sentences = [token for token in text.split('\n')]

In [11]:
tokens = []

database = {}

formatted_queries = [f'query: {sentence}' for sentence in sentences]

In [12]:
tokenized_sentences = tokenizer(
    formatted_queries, max_length=512,
    padding=True, truncation=True,
    return_tensors='pt'
)

In [13]:
tokenized_sentences['input_ids'] = tokenized_sentences['input_ids'].to(device)
tokenized_sentences['attention_mask'] = tokenized_sentences['attention_mask'].to(device)

In [14]:
from torch import Tensor

def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [17]:
from tqdm.auto import tqdm

model.eval()
outputs = []

with torch.no_grad():
    for i in tqdm(range(len(formatted_queries))):
        batch = {'input_ids': tokenized_sentences['input_ids'][i].unsqueeze(0), 'attention_mask': tokenized_sentences['attention_mask'][i].unsqueeze(0)}
        out = model(**batch)
        embeddings = average_pool(out.last_hidden_state, tokenized_sentences['attention_mask'][i].unsqueeze(0))
        outputs.append(embeddings.squeeze(0))

  0%|          | 0/704 [00:00<?, ?it/s]

In [18]:
outputs = torch.stack(outputs).detach().cpu()

In [19]:
outputs.shape

torch.Size([704, 1024])

In [20]:
import torch.nn.functional as F

embeddings = F.normalize(outputs, p=2, dim=1)

In [21]:
for i, sentence in enumerate(sentences):
    database[sentence] = embeddings[i]

In [22]:
import pickle

with open('embeddings_db.pkl', 'wb') as f:
    pickle.dump(database, f)

In [31]:
from typing import List


def rank(sentence: str, k: int=5) -> List[str]:
    model.eval()
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    sentence = f'query: {sentence}'
    tokenized_sentences = tokenizer(
        [sentence], max_length=512,
        padding=True, truncation=True,
        return_tensors='pt'
    )
    tokenized_sentences['input_ids'] = tokenized_sentences['input_ids'].to(device)
    tokenized_sentences['attention_mask'] = tokenized_sentences['attention_mask'].to(device)
    out = model(**tokenized_sentences)
    embeddings = average_pool(out.last_hidden_state, tokenized_sentences['attention_mask'])
    embeddings = embeddings.detach().cpu()
    search_metrics = {}

    for sentence in database.keys():
        metric = F.cosine_similarity(embeddings, database[sentence])
        search_metrics[sentence] = metric.item()

    sorted_scores = sorted(search_metrics.items(), key=lambda x: x[1], reverse=True)
    return [(sentence, score) for sentence, score in sorted_scores[:k]]

In [56]:
rank('расскажи про войну на украине 2022 году', 5)

[('21 февраля 2022 года Россия признала независимость ДНР и ЛНР, а 24 февраля своим полномасштабным вторжением на Украину эскалировала российско-украинскую войну до самой масштабной войны в Европе со времён Второй мировой. Российское вторжение привело к введению новых международных санкций.',
  0.8437685370445251),
 ('В октябре 2022 года Россия заявила об аннексии оккупированных ею частей Херсонской, Запорожской, Донецкой и Луганской областей Украины,.',
  0.8307884335517883),
 ('В 2022 году после вторжения российских войск на Украину МОК призвал международные спортивные федерации либо перенести, либо отменить любые спортивные мероприятия, запланированные в России или Белоруссии. Также он рекомендовал разрешить белорусским и российским спортсменам и командам соревноваться только в нейтральном статусе. В результате российских спортсменов отстранили от ряда международных спортивных соревнований, в том числе и от Лиги чемпионов УЕФА.',
  0.8239083886146545),
 ('«Война и мир» Сергея Бондар